In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense
import numpy as np
import pdb

In [2]:
filepath = '/Users/AnhVu/Study/PhD/mypaper/online_deep_forest/data/csv/electricity-normalized.csv'
dataset = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
print(dataset.shape)
X = dataset[:, :-1]
y = dataset[:, -1]

print(X.shape)
print(y.shape)

n_classes = np.unique(y).shape[0]
n_features = X.shape[1]
print('n_classes = ', n_classes)
print('n_features = ', n_features)

(45312, 9)
(45312, 8)
(45312,)
n_classes =  2
n_features =  8


In [3]:
class ONN(tf.keras.Model):
    def __init__(self, n_features, n_classes,
                n_hidden_units=10, beta=0.99,
                learning_rate=0.01, s=0.2,
                n_layers=20):
        super(ONN, self).__init__(name='ONN')
        self.n_features = n_features
        self.n_classes = n_classes
        self.n_hidden_units = n_hidden_units
        self.n_layers = n_layers
        self.beta = beta
        self.s = s
        
        self.hidden_layers = []
        self.output_layers = []
        
        for i in range(n_layers):
            self.hidden_layers.append(
                Dense(n_hidden_units)
            )
            
        for i in range(n_layers + 1):
            self.output_layers.append(
                Dense(n_classes)
            )
            
        self.alphas = tf.Variable(
            np.ones(n_layers + 1) * (1./(n_layers + 1)),
            dtype=tf.float32
        )
        
        
    def __call__(self, inputs):
        hidden_connections = []
        x = inputs
        hidden_connections.append(x)
        
        for i in range(self.n_layers):
            hidden_connections.append(
                tf.nn.relu(
                    self.hidden_layers[i](
                        hidden_connections[i]
                    )
                )
            )
        
        output_class = []
        for i in range(self.n_layers + 1):
            output_class.append(
                self.output_layers[i](
                    hidden_connections[i]
                )
            )
        
        return output_class
        

In [4]:
model = ONN(n_features, n_classes)
# plot_model(model, to_file='model.png')

v = tf.constant(X[0, :], shape=[1, n_features])
print(v)

print(len(model(v)))
print(model(v)[0])

tf.Tensor([[0.       1.       0.       0.056443 0.439155 0.003467 0.422915 0.414912]], shape=(1, 8), dtype=float32)
21
tf.Tensor([[-0.32111368 -0.26206058]], shape=(1, 2), dtype=float32)


In [5]:

def loss(model, x, y):
    output_class = model(x)
    losses = []
    for i in range(len(output_class)):
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        losses.append(
            loss_object(y_true=y, y_pred=output_class[i])
        )
        
    return losses

In [6]:
def train(model, inputs, targets, learning_rate):
    with tf.GradientTape(persistent=True) as tape:
        losses = loss(model, inputs, targets)
    
    # layer 0
    dV0_weight, dV0_bias = tape.gradient(
        losses[0], model.output_layers[0].variables
    )
    # update weight
    model.output_layers[0].variables[0].assign_sub(
        model.alphas[0] * learning_rate * dV0_weight
    )
    # update bias
    model.output_layers[0].variables[1].assign_sub(
        model.alphas[0] * learning_rate * dV0_bias
    )
    
    w = []
    b = []
    
    # layer 1,2,...,L
    for i in range(1, len(losses)):
        variables = []
        for j in range(i):
            variables.append(model.hidden_layers[j].variables)
            
        variables.append(model.output_layers[i].variables)
        
        grads = tape.gradient(
            losses[i], variables
        )
        
        # 1. For output layer
        dVi_weight, dVi_bias = grads[-1]
        
        # update weight
        model.output_layers[i].variables[0].assign_sub(
            model.alphas[i] * learning_rate * dVi_weight
        )
        # update bias
        model.output_layers[i].variables[1].assign_sub(
            model.alphas[i] * learning_rate * dVi_bias
        )
        
        # 2. For hidden layer
        for j in range(i):
            if len(w) < j + 1:
                w.append(tf.Variable(model.alphas[i] * grads[j][0]))
                b.append(tf.Variable(model.alphas[i] * grads[j][1]))
            else:
                w[j].assign_add(model.alphas[i] * grads[j][0])
                b[j].assign_add(model.alphas[i] * grads[j][1])
        
    for i in range(len(losses) - 1):
        model.hidden_layers[i].variables[0].assign_sub(
            learning_rate * w[i]
        )
        
        model.hidden_layers[i].variables[1].assign_sub(
            learning_rate * b[i]
        )
    
    for i in range(len(losses)):
        model.alphas[i].assign(
            tf.math.pow(
                model.beta, losses[i]
            )
        )
        
        model.alphas[i].assign(
            tf.math.maximum(
                model.alphas[i],
#                 model.s / model.n_layers
                0.01
            )
        )
    
    z_t = tf.math.reduce_sum(model.alphas)
    model.alphas.assign(model.alphas / z_t)
    


In [18]:
model = ONN(n_features, n_classes, n_layers=0)

# inputs = tf.constant(X[:, :], shape=[X.shape[0], n_features])
# outputs = tf.constant(y[:], shape=[y.shape[0]])
epochs = range(1)
cnt = 0
for epoch in epochs:
    for i in range(X.shape[0]):

        inputs = tf.constant(X[i, :], shape=[1, n_features])
        
        outputs = model(inputs)
        prob = np.zeros(n_classes)
        for k in range(len(outputs)):
            t = model.alphas[k].numpy() * tf.nn.softmax(outputs[k])
            prob = prob + t.numpy()
        
        preds = np.argmax(prob, axis=1)
        pred = preds[0]
        if pred == y[i]:
            cnt += 1
            
        if (i % 100 == 0):
            print('#{}'.format(i))
            print(cnt / (i + 1))
        
        targets = tf.constant(y[i], shape=[1])
        
        current_loss = loss(model, inputs, targets)

        train(model, inputs, targets, learning_rate=0.01)

y_pred = model(inputs).numpy()
print(y_pred.shape)

#0
1.0
#100
0.38613861386138615
#200
0.5174129353233831


KeyboardInterrupt: 

In [ ]:
a, b = [1,2]
print(a,b)